In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import partial

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.linear_model import OLS
from linearmodels import PanelOLS
from linearmodels.iv.model import IV2SLS
from linearmodels.panel.results import compare

data_dir = "/Users/mac/Desktop/Study/Diploma/data"

# Filtering Countries

In [2]:
def dummy_country(x, code):
    return 1 * np.any(x == code)

def return_unique(x):
    return len(x.unique())

In [3]:
PARTNERS = 25
gtd_path = os.path.join(data_dir, "gtd/gtd_processed")
WITS_path = os.path.join(data_dir, "countries", "WITS_codes.xlsx")

df = pd.read_parquet(os.path.join(gtd_path, "gtd2005.parquet"))
df.columns = [item.lower() for item in df.columns]

WITS_df = pd.read_excel(WITS_path).drop(columns="ISO3")

In [4]:
top_countries = df.groupby("code").agg({"inn": return_unique})\
                .sort_values(by="inn", ascending=False).iloc[:PARTNERS]\
                .merge(WITS_df, on="code").loc[:,["code", "country"]]

In [5]:
years= [2005, 2006, 2007, 2008, 2009]

result = top_countries
for year in tqdm(years):
    item_df = pd.read_parquet(os.path.join(gtd_path, f"gtd{year}.parquet"))
    item_df.columns = [item.lower() for item in item_df.columns]
    
    item_df = item_df.groupby("code").agg({"inn": return_unique})\
            .rename(columns={"inn": "partners_{}".format(year)})
    
    result = result.merge(item_df, on="code", how="inner")

result

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00,  7.30it/s]


,code,country,partners_2005,partners_2006,partners_2007,partners_2008,partners_2009
0,398,Kazakhstan,8999,9519,10016,9968,11018
1,804,Ukraine,5902,6006,5871,5915,5468
2,156,China,2937,2735,2541,2195,2694
3,276,Germany,2625,2625,2723,2696,2984
4,440,Lithuania,2465,2382,2185,1890,1689
5,860,Uzbekistan,2428,2417,2744,2784,3143
6,428,Latvia,2351,2215,2176,1842,1620
7,233,Estonia,1958,1849,1628,1128,1125
8,31,Azerbaijan,1877,1879,2137,2258,2417
9,246,Finland,1738,1592,1396,1269,1393


In [6]:
top_cty_list = top_countries.code.unique()
funcs = [
    (f"{code}", partial(dummy_country, code=code))
    for code in top_cty_list
]

result_raw = []
for year in tqdm(years):
    item_df = pd.read_parquet(os.path.join(gtd_path, f"gtd{year}.parquet"))
    item_df.columns = [item.lower() for item in item_df.columns]
    
    item_df = item_df.groupby("inn").agg({"code": funcs})\
                .assign(year=year)

    item_df.columns = ["_".join(item).rstrip("_") for item in item_df]
    
    result_raw.append(item_df)

result_raw = pd.concat(result_raw).reset_index()

100%|██████████| 5/5 [01:29<00:00, 17.84s/it]


In [7]:
result = []
years_df = pd.DataFrame({"year": years})
for _, item_df in tqdm(result_raw.groupby("inn")):
    item_df = item_df.merge(years_df, on="year")
    item_years = item_df[["inn", "year"]]
    item_df = item_df.sort_values(by=["year"]).drop(columns=["year", "inn"]).diff(1)
    result.append(pd.concat([item_years, item_df], axis=1))

result = pd.concat(result).replace(-1, 0).dropna()

100%|██████████| 68052/68052 [00:42<00:00, 1612.94it/s]


In [8]:
result.replace(-1, 0).dropna().groupby("year").sum().drop(columns="inn").T.astype(int)

year,2006,2007,2008,2009
code_398,886,949,949,903
code_804,813,873,898,546
code_156,271,311,297,403
code_276,550,619,618,522
code_440,515,486,436,335
code_860,614,766,789,673
code_428,485,516,442,323
code_233,413,340,273,256
code_31,483,590,642,505
code_246,200,214,214,146


In [9]:
result.to_parquet(os.path.join(data_dir, "gtd/customs_advanced.parquet"), index=False)

# Instrument Preparation

In [10]:
iv_path = os.path.join(data_dir, "instrument/iv.parquet")

iv_df = pd.read_parquet(iv_path)\
            .assign(instrument=lambda x: x["weight_c"] * x["tariff"] / 100)\
            .merge(top_countries, on=["code"], how="inner")\
            .groupby(["okved_four", "year", "code"]).agg({"instrument": "sum"})\
            .reset_index()

print(len(iv_df))

iv_df = iv_df.pivot(index=["okved_four", "year"], columns="code", values="instrument").reset_index().fillna(0)

iv_df.columns = ["iv_{}".format(item) if isinstance(item, int) else item for item in iv_df.columns]

iv_df.head()

27100


,okved_four,year,iv_31,iv_51,iv_100,iv_156,iv_233,iv_246,iv_250,iv_268,...,iv_440,iv_498,iv_528,iv_616,iv_762,iv_792,iv_804,iv_826,iv_840,iv_860
0,01.11,2005,0.035459,0.1,0.0,0.023226,0.003117,0.0,0.0,0.118567,...,0.0,0.1,0.0,0.0,0.0,0.071524,0.057462,0.0,0.0,0.0
1,01.11,2006,0.035459,0.1,0.0,0.023226,0.002628,0.0,0.0,0.118567,...,0.0,0.1,0.0,0.0,0.0,0.071524,0.058576,0.0,0.0,0.0
2,01.11,2007,0.035459,0.1,0.0,0.023226,0.002631,0.0,0.0,0.015992,...,0.0,0.1,0.0,0.0,0.0,0.071524,0.069732,0.0,0.0,0.0
3,01.11,2008,0.035459,0.1,0.0,0.023096,0.002631,0.0,0.0,0.015992,...,0.0,0.1,0.0,0.0,0.0,0.071524,0.070808,0.0,0.0,0.0
4,01.11,2009,0.035459,0.1,0.0,0.023226,0.002632,0.0,0.0,0.015992,...,0.0,0.1,0.0,0.0,0.0,0.071524,0.068341,0.0,0.0,0.0


In [11]:
iv_df.to_parquet(os.path.join(data_dir, "instrument/iv_advanced.parquet"), index=False)

# Data Preparation

In [2]:
def return_unique(x):
    return len(x.unique())

top_cty_list = [398, 804, 156, 276, 440, 860, 428, 233,  31, 246, 498, 840, 616,
       417, 268, 380, 792, 392,  51, 762, 528, 826, 410, 100, 250]

In [3]:
spark_path = os.path.join(data_dir, "spark/nxt_spark_data.parquet")
ruslana_path = os.path.join(data_dir, "ruslana/ruslana.parquet")
gtd_path = os.path.join(data_dir, "gtd/gtd_processed")
iv_path = os.path.join(data_dir, "instrument/iv_advanced.parquet")
gtd_advanced_path = os.path.join(data_dir, "gtd/customs_advanced.parquet")

os.listdir(gtd_path)
spark_df = pd.read_parquet(spark_path)

spark_df.columns = [item.lower() for item in spark_df.columns]
            
spark_df = spark_df.loc[~spark_df["okved_four"].isin(['nan', 'None'])]
iv_df = pd.read_parquet(iv_path)

tables = os.listdir(gtd_path)
gtd_df = []
for table in tables:
    df = pd.read_parquet(os.path.join(gtd_path, table))
    df = df.loc[df["INN"] > 100]\
            .groupby(["INN", "year"]).agg({"code": return_unique})\
            .reset_index().rename(columns={"code": "num_countries"})
    gtd_df.append(df)
    
gtd_df = pd.concat(gtd_df)
gtd_df.columns = [item.lower() for item in gtd_df.columns]

ruslana_df = pd.read_parquet(ruslana_path)

print(len(spark_df), len(gtd_df), len(ruslana_df), len(iv_df))

df = spark_df.merge(ruslana_df, on=["inn", "year"], how="inner")
print(len(df))
df = df.merge(gtd_df, on=["inn", "year"], how="left")
df = df.merge(iv_df, on=["okved_four", "year"], how="inner")
print(len(df))

df = df.drop_duplicates(["inn", "year"])

print(len(df))

2335164 139420 3946725 2373
1858619
1785827
1305256


In [30]:
filter_cond = (df.assets > 0.)

data = df.loc[filter_cond]\
        .sort_values(by=["inn", "year"])\
        .assign(
            short_leverage=lambda x: x.short_debt / x.assets, 
            long_leverage=lambda x: x.long_debt / x.assets, 
            leverage=lambda x: x.debt / x.assets, 
            log_assets=lambda x: np.log(x.assets),
            tangibility=lambda x: x.tang_assets / x.assets, 
            profitability=lambda x: x.revenue / x.assets
        )

print("Assets more then 0: {}".format(len(data)))

filter_cond = (
    (data["short_leverage"] >= 0.) &
    (data["long_leverage"] >= 0.) &
    (data["leverage"] <= 1.) &
    (data.revenue > 0.0) &
    (data.assets >= 2100) &
    (data.assets < 871880628) &
    (data.empl >= 2.0)
)

data = data.loc[filter_cond]
print("Employees no less than 5: {}".format(len(data)))

years = pd.DataFrame(np.arange(2004, 2010), columns=["year"])

export_data = []
for _, item_df in tqdm(data.loc[~data.num_countries.isnull()].groupby("inn")):
    item_df = item_df.merge(years, on="year", how="right")\
                .assign(
                    num_countries=lambda x: x.num_countries.fillna(0.0), 
                    num_countries_prev=lambda x: x.num_countries.shift(1),
                ).dropna(subset=["inn"])

    export_data.append(item_df)

export_data = pd.concat(export_data)\
        .assign(
            countries_diff=lambda x: x.num_countries - x.num_countries_prev,
        )[["inn", "year", "num_countries_prev", "countries_diff"]]

data = data.merge(export_data, on=["inn", "year"], how="left")\
        .assign(
            num_countries_prev=lambda x: x.num_countries_prev.fillna(0.0),
            num_countries_prev_log=lambda x: np.log(1 + x.num_countries_prev),
        )

data = data[data.year > 2005]

print(len(data))

customs_advanced = pd.read_parquet(gtd_advanced_path)
data = data.merge(customs_advanced, on=["inn", "year"], how="left")\
            .fillna(0)

data.head()

Assets more then 0: 1300603
Employees no less than 5: 943763


100%|██████████| 14750/14750 [00:16<00:00, 910.49it/s]


743231


,inn,okved,year,tang_assets,assets,short_debt,revenue,opex,profit,long_debt,...,code_380,code_792,code_392,code_51,code_762,code_528,code_826,code_410,code_100,code_250
0,101000021,47.73,2007,176000.0,186000.0,82000.0,457000.0,0.0,18000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,101000021,47.73,2008,308000.0,318000.0,95000.0,1943000.0,0.0,76000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,101000078,74.20,2006,896000.0,1131000.0,1000.0,2568000.0,0.0,314000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101000078,74.20,2007,1146000.0,1618000.0,1000.0,3971000.0,0.0,542000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,101000078,74.20,2008,1492000.0,2022000.0,1000.0,6211000.0,0.0,487000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Сохраним данные для Stata
TO_DROP = [
    "inn",
    "tang_assets",
    "assets",
    "short_debt",
    "revenue",
    "opex",
    "profit",
    "long_debt",
    "debt",
    "empl",
    "num_countries",
    "num_countries_prev",
    "countries_diff",
    "okved"
]
to_study = data.assign(exporting=lambda x: 1 * (x["num_countries"] > 0))\
    .dropna()

inns = to_study["inn"].unique()

inns = pd.DataFrame(inns, columns=["inn"])\
        .reset_index().rename(columns={"index": "firm_id"})

to_study = to_study.merge(inns, on=["inn"], how="inner")\
            .drop(columns=TO_DROP)
print(len(to_study))

to_study.to_csv(os.path.join(data_dir, "testing/cur_spark_ruslana_advanced_test.csv"), index=False)

743231


In [51]:
to_study.columns

Index(['year', 'okved_four', 'iv_31', 'iv_51', 'iv_100', 'iv_156', 'iv_233',
       'iv_246', 'iv_250', 'iv_268', 'iv_276', 'iv_380', 'iv_392', 'iv_398',
       'iv_410', 'iv_417', 'iv_428', 'iv_440', 'iv_498', 'iv_528', 'iv_616',
       'iv_762', 'iv_792', 'iv_804', 'iv_826', 'iv_840', 'iv_860',
       'short_leverage', 'long_leverage', 'leverage', 'log_assets',
       'tangibility', 'profitability', 'num_countries_prev_log', 'code_398',
       'code_804', 'code_156', 'code_276', 'code_440', 'code_860', 'code_428',
       'code_233', 'code_31', 'code_246', 'code_498', 'code_840', 'code_616',
       'code_417', 'code_268', 'code_380', 'code_792', 'code_392', 'code_51',
       'code_762', 'code_528', 'code_826', 'code_410', 'code_100', 'code_250',
       'exporting', 'firm_id'],
      dtype='object')

# Регрессии

In [27]:
top_cty_list = [398, 804, 156, 276, 440, 860, 428, 233,  31, 246, 498, 840, 616,
       417, 268, 380, 792, 392,  51, 762, 528, 826, 410, 100, 250]

In [53]:
data_path = os.path.join(data_dir, "testing/cur_spark_ruslana_advanced_test.csv")

data = pd.read_csv(data_path)

singletons = data.groupby("firm_id").agg({"year": "count"})
singletons = singletons[singletons["year"] > 1].index

data = data[data["firm_id"].isin(singletons)]

In [54]:
target = "code_840"
reg = "iv_840"
controls = ['log_assets', 'tangibility', 'profitability', 'num_countries_prev_log']

cols = [reg] + controls
to_study = data.set_index(['firm_id', 'year'])[cols + [target, "leverage"]].dropna(subset=cols + [target, "leverage"])

exog = sm.add_constant(to_study[cols])
model = PanelOLS(to_study[target], exog, entity_effects=True)
result = model.fit(cov_type='robust')

result.summary

Dep. Variable:,code_840,R-squared:,1.259e-05
Estimator:,PanelOLS,R-squared (Between):,0.0017
No. Observations:,581075,R-squared (Within):,1.259e-05
Date:,"Sat, May 10 2025",R-squared (Overall):,0.0007
Time:,11:39:24,Log-likelihood,1.675e+06
Cov. Estimator:,Robust,,
,,F-statistic:,0.8986
Entities:,224203,P-value,0.4808
Avg Obs:,2.5917,Distribution:,"F(5,356867)"
Min Obs:,2.0000,,
Max Obs:,3.0000,F-statistic (robust):,0.7950


In [28]:
target = "leverage"
regs = ["code_{}".format(item) for item in top_cty_list]
controls = ['log_assets', 'tangibility', 'profitability', 'num_countries_prev_log']

cols = regs + controls
to_study = data.set_index(['firm_id', 'year'])[cols + [target]].dropna(subset=cols + [target])

exog = sm.add_constant(to_study[cols])
model = PanelOLS(to_study[target], exog, entity_effects=True)
result = model.fit(cov_type='robust')

result.summary

Dep. Variable:,leverage,R-squared:,0.1365
Estimator:,PanelOLS,R-squared (Between):,-0.1731
No. Observations:,581075,R-squared (Within):,0.1365
Date:,"Thu, May 08 2025",R-squared (Overall):,-0.1478
Time:,18:02:18,Log-likelihood,4.856e+05
Cov. Estimator:,Robust,,
,,F-statistic:,1945.7
Entities:,224203,P-value,0.0000
Avg Obs:,2.5917,Distribution:,"F(29,356843)"
Min Obs:,2.0000,,
Max Obs:,3.0000,F-statistic (robust):,625.49


In [49]:
for item in tqdm(top_cty_list):
    target = "code_{}".format(item)
    reg = "iv_{}".format(item)
    controls = ['log_assets', 'tangibility', 'profitability', 'num_countries_prev_log']

    cols = [reg] + controls
    to_study = data.set_index(['firm_id', 'year'])[cols + [target]]

    exog = sm.add_constant(to_study[cols])
    model = PanelOLS(to_study[target], exog, entity_effects=True, time_effects=True)
    res = model.fit(cov_type='robust')
    data = data.assign(**{"pred_{}".format(item): res.predict(exog).predictions.values})

100%|██████████| 25/25 [00:43<00:00,  1.74s/it]


In [50]:
target = "leverage"
regs = ["pred_{}".format(item) for item in top_cty_list]
controls = ['log_assets', 'tangibility', 'profitability', 'num_countries_prev_log']

cols = regs + controls
to_study = data.set_index(['firm_id', 'year'])[cols + [target]].dropna(subset=cols + [target])

exog = sm.add_constant(to_study[cols])
model = PanelOLS(to_study[target], exog, entity_effects=True)
result = model.fit(cov_type='robust')

result.summary

Dep. Variable:,leverage,R-squared:,0.1570
Estimator:,PanelOLS,R-squared (Between):,-1.3714
No. Observations:,581075,R-squared (Within):,0.1570
Date:,"Thu, May 08 2025",R-squared (Overall):,-1.2292
Time:,18:45:56,Log-likelihood,4.926e+05
Cov. Estimator:,Robust,,
,,F-statistic:,2292.5
Entities:,224203,P-value,0.0000
Avg Obs:,2.5917,Distribution:,"F(29,356843)"
Min Obs:,2.0000,,
Max Obs:,3.0000,F-statistic (robust):,770.38
